### from : rag_cot_peactice.md

## 환경 정보

In [1]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

# .env 로드
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_LLM_MODEL = os.getenv("OPENAI_LLM_MODEL")  # ex: "gpt-4o-mini"

# LLM 초기화: n개의 샘플 생성
llm = ChatOpenAI(
    model_name=OPENAI_LLM_MODEL,
    openai_api_key=OPENAI_API_KEY,
    temperature=0.7,
    # temperature=0.0,
    n=5
)

## AnswerParser

In [2]:
from langchain.schema import Document
from langchain.output_parsers import RegexParser

# 가정: 이미 Retriever가 Top-3 문서를 가져옴
docs = [
    Document(page_content="파리는 프랑스의 수도로, 연간 관광객 수 약 3000만 명입니다."),
    Document(page_content="런던은 영국의 수도로, 연간 관광객 수 약 2000만 명입니다."),
    Document(page_content="교토는 일본의 옛 수도로, 연간 관광객 수 약 1500만 명입니다.")
]


### 난이도 하 (Easy)

In [5]:
from langchain.prompts import PromptTemplate

# 예시 두 개를 포함하는 Few-Shot
template = PromptTemplate.from_template(
    '''문서:
{docs}

예시:
Q: 파리의 국가는?
A: 프랑스

Q: 런던의 국가는?
A: 영국

Q: {query}
A:'''  
)
parser = RegexParser(regex=r'(?s)(.*)', output_keys=['answer'])
chain = template | llm | parser
res = chain.invoke({
    'docs': '\n\n'.join(doc.page_content for doc in docs),
    'query': '교토의 국가는?'
})
print(res['answer'])

일본


### 난이도 중 (Medium)

In [21]:
import json
# 두 개의 JSON Q-A 예시 포함
template = PromptTemplate.from_template(
    '''문서:
{docs}

예시:
Q: 파리의 관광객 수는?
A: [{{"city":"파리","visitors":"3000만"}}]

Q: 런던의 관광객 수는?
A: [{{"city":"런던","visitors":"2000만"}}]

Q: {query}
A:
응답은 JSON 배열만 출력하세요.
'''  
)
parser = RegexParser(regex=r'(\[.*\])', output_keys=['json_str'])
# chain = template | llm
chain = template | llm | parser
res = chain.invoke({
    'docs': '\n\n'.join(doc.page_content for doc in docs),
    'query': '교토의 관광객 수는?'
})
print(res)
data = json.loads(res['json_str'])
print(data)

{'json_str': '[{"city":"교토","visitors":"1500만"}]'}
[{'city': '교토', 'visitors': '1500만'}]


### 난이도 상 (Hard)

In [12]:
# 세 개의 JSON 예시 포함 (조건부)
template = PromptTemplate.from_template(
    '''문서:
{docs}

예시:
Q: 파리 연간 성장률은?
A: [{{"year":"2023","growth":"2.5%"}}]

Q: 런던 연간 성장률은?
A: [{{"year":"2023","growth":"1.8%"}}]

Q: {query}
응답은 JSON 배열만 출력하세요. 데이터 없으면 빈 배열.
A:'''  
)
parser = RegexParser(regex=r'(\[.*\])', output_keys=['json_str'])
chain = template | llm | parser
res = chain.invoke({
    'docs': '\n\n'.join(doc.page_content for doc in docs),
    # 'query': '파리 연간 성장률은?',   # 데이터 있음
    'query': '교토 연간 성장률은?',     # 데이터 없으면 빈 배열
})
data = json.loads(res['json_str'])
print(data)

[]
